In [47]:
#Caricare le librerie
import tweepy
import pandas as pd
import numpy as np
import re
import json

In [48]:
consumer_key= "rrR0Znz98oKkjfB8b72oLH9Qc"
consumer_secret= "m7iOPX6iW8R3IalRf7x56YVeIb18o3I1bhUftg8JzlQ6fAjLgW"
access_token= "1442250607-3cJYKYyh7Ae19nMcv5YoxOIB2XlvgVX0ROaDSJx"
access_token_secret= "ZOnaaCTgO7DsKtmPjVruOlkvzBY8p6MbkKLjiYLdamBYE"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [55]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")
#NB: hasattr(obj, name, /) Return whether the object has an attribute with the given name.
hashtag="#lockdown"
list_tweet=[]
list_originals=[]
list_rt_id=[]
list_retweetted=[]
n=4500
for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,count=n,lang="it").items(n):

  #gestione dei retweet con try-except
  try:
    list_originals.append(tweet.retweeted_status.id) #creo lista dei retweets id 
    list_rt_id.append(tweet.retweeted_status.user.id)

    #lista tweets che sono stati retweettati
    list_retweetted.append([tweet.retweeted_status.created_at, tweet.retweeted_status.id,tweet.retweeted_status.user.id, tweet.retweeted_status.text, 
                     tweet.retweeted_status.favorite_count, tweet.retweeted_status.retweet_count, 
                     tweet.retweeted_status.user.screen_name]) 
  except:
    list_originals.append(0)
    list_rt_id.append(0)
  #creo lista e df dei tweet  
  list_tweet.append([tweet.created_at, tweet.id,tweet.user.id, tweet.text, 
                     tweet.favorite_count, tweet.retweet_count, 
                     tweet.user.location,tweet.user.screen_name,
                     hasattr(tweet, "retweeted_status"),
                     tweet.in_reply_to_screen_name])

                     
tweets = pd.DataFrame(list_tweet[1:], 
                      columns=['date','id','user_id','text','like','n_rt','location','author',
                               "retweet","reply_name"])

tweets["id_retweet"]=list_originals[1:]
tweets["user_id_retweet"]=list_rt_id[1:]
#creo df dei retweet
retweetted = pd.DataFrame(list_retweetted[1:], 
                      columns=['date','id','author_id','text','like','n_rt','author'])                                                    
tweets.head()

,date,id,user_id,text,like,n_rt,location,author,retweet,reply_name,id_retweet,user_id_retweet
0,2021-12-24 17:13:55+00:00,1474428132858146820,1535934018,RT @lllll66099563: Vi faccio ridere. Su Facebo...,0,1,Sardegna,samdcarta,True,None,1474427338637119495,1397552279738273793
1,2021-12-24 17:11:50+00:00,1474427610969038851,2713176561,RT @christian_fsi: Non mi metto nei panni del ...,0,14,MILANO,AssiElena,True,None,1474102780117803015,734134273
2,2021-12-24 17:10:45+00:00,1474427338637119495,1397552279738273793,Vi faccio ridere. Su Facebook è pieno di taleb...,5,1,,lllll66099563,False,None,0,0
3,2021-12-24 17:05:38+00:00,1474426051694575616,225490204,16.000 e passa solo in #Lombardia.\n\n#Lockdow...,0,0,,call_the_kernel,False,None,0,0
4,2021-12-24 17:01:04+00:00,1474424900047802377,1112753621601521669,RT @Willhel77324546: Molti sono preoccupati e ...,0,3,,Graziel69066979,True,None,1474272149582102529,1412670347095293954


In [56]:
#selezione dei tweet più retwettati 
retweetted.sort_values("n_rt",ascending=False).drop_duplicates()

,date,id,author_id,text,like,n_rt,author
1032,2021-11-19 13:28:48+00:00,1461687908797227008,50989769,Mi sembra il giorno giusto per ripostare il fa...,2479,1331,darkskywriter
1214,2021-12-20 15:47:47+00:00,1472956905664520192,2222183850,Noi invece Proponiamo il #Lockdown TOTALE e de...,901,677,EsercitoCrucian
1327,2021-12-18 19:09:39+00:00,1472282934073872397,3096871079,Che fate a Capodown?\n#lockdown,4775,672,lefrasidiosho
255,2021-12-18 22:56:11+00:00,1472339940327444482,2377487094,Con il 90% di #vaccinati si moltiplicano i pos...,2241,594,intuslegens
280,2021-12-18 20:23:00+00:00,1472301392467476489,1323040160,Saranno intitolate strade e piazze in suo onor...,1413,484,strange_days_82
...,...,...,...,...,...,...,...
2541,2021-12-20 17:24:52+00:00,1472981336763871233,1467622232935571457,La mia libertà non finisce dove cominciano le ...,6,1,TomUbickV_V
2542,2021-12-20 17:25:02+00:00,1472981382695641105,1467954714788700169,Nessuno ci può togliere la libertà di dire NO....,5,1,VivoLibero1
2544,2021-12-20 17:25:15+00:00,1472981433681686535,1467094064449441794,Questo è nazismo criminale! Atti insopportabil...,0,1,Nobu_ViVi
2545,2021-12-20 17:25:15+00:00,1472981436345036805,1467622232935571457,Se dici una bugia abbastanza grande e continui...,5,1,TomUbickV_V


##### Vado a vedere quali sono i tweet più retwittati e quante volte sono stati ritwittati dalla nostra rete (usando SQL)

In [57]:
#Open the terminal and run-   pip install U pandasql
#Open your Jupyter Notebook and in any cell run-    !pip install -U pandasql
import pandasql as ps
q1 = """SELECT id,like,n_rt, author,count() FROM retweetted group by id order by n_rt desc"""
print(ps.sqldf(q1, locals()))

                      id  like  n_rt           author  count()
0    1461687908797227008  2479  1331    darkskywriter        1
1    1472956905664520192   901   677  EsercitoCrucian      522
2    1472282934073872397  4775   672    lefrasidiosho        9
3    1472339940327444482  2241   594      intuslegens       10
4    1472301392467476489  1413   484  strange_days_82       12
..                   ...   ...   ...              ...      ...
342  1472854224719552514     0     1   InformazioneOg        1
343  1472638825369382913     2     1    ContiMariapia        1
344  1472587980284043276    10     1  GiorgioCameran1        1
345  1472458878268985346     1     1        Radio1Rai        1
346  1470810332654362634     2     1   BorasoGianluca        1

[347 rows x 5 columns]


#### Creazione input Grephi tramite scrittura file .csv
Da rivedere:
- Abbiamo creato nodi con l'id del tweet, non per utente
- Si possono pesare gli archi tra utenti per il numero di retweet tra essi? Ha senso?

In [58]:
rt_tweets=tweets[tweets.retweet==True]

In [59]:
tweets_ids=rt_tweets[['user_id','author']]
retwitted_ids=retweetted[['author_id','author']].rename(columns={'author_id':'user_id'})
rt_tweets_ids=pd.concat([tweets_ids,retwitted_ids]).drop_duplicates().reset_index(drop=True)

In [60]:
Nodes_df=pd.DataFrame({'Id':rt_tweets_ids.user_id,'Label':rt_tweets_ids.author,'Interval':None})

In [61]:
Edges_df=pd.DataFrame({'Source':rt_tweets.user_id,'Target':rt_tweets.user_id_retweet,'Type':'Directed','Kind':None,'Id':range(1, len(rt_tweets)+1),'Label':None,'Interval':None,'Weight':1.0})

In [62]:
Nodes_df.to_csv('Nodes_df.csv', index=False)
Edges_df.to_csv('Edges_df.csv', index=False)

##### Grafi con NetworkX

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
Edges_df=pd.read_csv("C:/Users/dvd_m/Desktop/Social Media Analytics/SMA-Project/Edges_df.csv")
G = nx.from_pandas_edgelist(Edges_df, source='Source', target='Target')
df_cluster=pd.DataFrame({"nodes":nx.clustering(G).keys(),"cluster":nx.clustering(G).values()})
grafo=pd.merge(Edges_df,df_cluster, left_on="Source",right_on="nodes")